### Carregando o DataFrame tratado

In [1]:
import pandas as pd

df = pd.read_csv('TelecomX_data_tratados.csv')

print("Colunas originais antes da remoção:")
print(df.columns)
print("\nPrimeiras 5 linhas do DataFrame original:")
print(df.head())

# Identificando e eliminando colunas que não agregam valor preditivo
# 'ID_Cliente' é um identificador único e geralmente não é útil para prever churn.
# 'Unnamed: 0' é uma coluna de índice que pode ter sido gerada ao salvar o CSV, e também não é útil.
colunas_para_eliminar = ['ID_Cliente', 'Unnamed: 0'] 

# Verificando se as colunas existem antes de tentar eliminá-las para evitar erros
colunas_existentes_para_eliminar = [col for col in colunas_para_eliminar if col in df.columns]

if colunas_existentes_para_eliminar:
    df = df.drop(columns=colunas_existentes_para_eliminar)
    print(f"\nColunas eliminadas: {colunas_existentes_para_eliminar}")
else:
    print("\nNenhuma das colunas especificadas para eliminação foi encontrada.")

print("\nColunas após a eliminação:")
print(df.columns)
print("\nPrimeiras 5 linhas do DataFrame após a eliminação:")
print(df.head())

Colunas originais antes da remoção:
Index(['ID_Cliente', 'Cancelou', 'Genero', 'Idoso', 'Possui_Conjuge',
       'Possui_Dependentes', 'Meses_Permanencia', 'Servico_Telefone',
       'Multiplas_Linhas', 'Tipo_Internet', 'OnlineSecurity', 'Backup_Online',
       'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV',
       'Streaming_Filmes', 'Tipo_Contrato', 'Fatura_Digital',
       'Metodo_Pagamento', 'Cobranca_Mensal', 'Cobranca_Total'],
      dtype='object')

Primeiras 5 linhas do DataFrame original:
   ID_Cliente Cancelou     Genero  Idoso Possui_Conjuge Possui_Dependentes  \
0  0002-ORFBO      Não   Feminino      0            Sim                Sim   
1  0003-MKNFE      Não  Masculino      0            Não                Não   
2  0004-TLHLJ      Sim  Masculino      0            Não                Não   
3  0011-IGKFF      Sim  Masculino      1            Sim                Não   
4  0013-EXCHZ      Sim   Feminino      1            Sim                Não   

   Meses_Permanenc

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Cancelou              7043 non-null   object 
 1   Genero                7043 non-null   object 
 2   Idoso                 7043 non-null   int64  
 3   Possui_Conjuge        7043 non-null   object 
 4   Possui_Dependentes    7043 non-null   object 
 5   Meses_Permanencia     7043 non-null   int64  
 6   Servico_Telefone      7043 non-null   object 
 7   Multiplas_Linhas      7043 non-null   object 
 8   Tipo_Internet         7043 non-null   object 
 9   OnlineSecurity        7043 non-null   object 
 10  Backup_Online         7043 non-null   object 
 11  Protecao_Dispositivo  7043 non-null   object 
 12  Suporte_Tecnico       7043 non-null   object 
 13  Streaming_TV          7043 non-null   object 
 14  Streaming_Filmes      7043 non-null   object 
 15  Tipo_Contrato        

### Transformando váriaveis categóricas em formato numérico

In [4]:
# CORREÇÃO: Transformar a coluna 'Cancelou' para numérica antes de qualquer encoding
df['Cancelou'] = df['Cancelou'].replace({'Sim': 1, 'Não': 0})
df.dropna(subset=['Cancelou'], inplace=True)
df['Cancelou'] = df['Cancelou'].astype(int)

# Identificando as colunas categóricas para o one-hot encoding
colunas_categoricas = df.select_dtypes(include='object').columns

# Realizando o One-Hot Encoding
# O parâmetro 'columns' recebe a lista de colunas a serem transformadas.
# O 'drop_first=True' é uma boa prática para evitar multicolinearidade (uma das colunas criadas é redundante).
df_encoded = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True)

print("Colunas do DataFrame original:")
print(df.columns.tolist())
print("\nColunas do DataFrame após o One-Hot Encoding:")
print(df_encoded.columns.tolist())
print("\nPrimeiras 5 linhas do DataFrame após a transformação:")
print(df_encoded.head())

Colunas do DataFrame original:
['Cancelou', 'Genero', 'Idoso', 'Possui_Conjuge', 'Possui_Dependentes', 'Meses_Permanencia', 'Servico_Telefone', 'Multiplas_Linhas', 'Tipo_Internet', 'OnlineSecurity', 'Backup_Online', 'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV', 'Streaming_Filmes', 'Tipo_Contrato', 'Fatura_Digital', 'Metodo_Pagamento', 'Cobranca_Mensal', 'Cobranca_Total']

Colunas do DataFrame após o One-Hot Encoding:
['Cancelou', 'Idoso', 'Meses_Permanencia', 'Cobranca_Mensal', 'Cobranca_Total', 'Genero_Masculino', 'Possui_Conjuge_Sim', 'Possui_Dependentes_Sim', 'Servico_Telefone_Sim', 'Multiplas_Linhas_Sem serviço telefônico', 'Multiplas_Linhas_Sim', 'Tipo_Internet_Fibra ótica', 'Tipo_Internet_Não', 'OnlineSecurity_Sem serviço de internet', 'OnlineSecurity_Sim', 'Backup_Online_Sem serviço de internet', 'Backup_Online_Sim', 'Protecao_Dispositivo_Sem serviço de internet', 'Protecao_Dispositivo_Sim', 'Suporte_Tecnico_Sem serviço de internet', 'Suporte_Tecnico_Sim', 'Streamin

C:\Users\Suporte\AppData\Local\Temp\ipykernel_2548\4250839113.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Cancelou'] = df['Cancelou'].replace({'Sim': 1, 'Não': 0})


### Verificando a proporção de evasão

In [5]:
# Separando as features (X) e a variável alvo (y)
X = df_encoded.drop('Cancelou', axis=1)
y = df_encoded['Cancelou']

# --- Calculando a proporção de clientes que evadiram (parte da sua solicitação) ---

# Calculando a contagem de clientes para cada classe (Evadiram e Ativos)
contagem_classes = y.value_counts()

# Calculando a proporção em porcentagem
proporcao_classes = y.value_counts(normalize=True) * 100

print("\n" + "="*50)
print("Análise do Desequilíbrio de Classes:")
print("="*50)
print("Contagem de clientes por classe:")
print(f"Ativos (0): {contagem_classes[0]} clientes")
print(f"Evadiram (1): {contagem_classes[1]} clientes")

print("\nProporção das classes:")
print(f"Ativos (0): {proporcao_classes[0]:.2f}%")
print(f"Evadiram (1): {proporcao_classes[1]:.2f}%")

# Avaliando o desequilíbrio de classes
if proporcao_classes[1] < 30:
    print("\nAVISO: O desequilíbrio de classes é significativo.")
    print("A classe minoritária (Evadiram) representa menos de 30% do total de clientes.")
    print("Isso pode afetar o desempenho dos modelos preditivos, fazendo-os tender a prever a classe majoritária.")
else:
    print("\nAs classes estão relativamente balanceadas.")


Análise do Desequilíbrio de Classes:
Contagem de clientes por classe:
Ativos (0): 5174 clientes
Evadiram (1): 1869 clientes

Proporção das classes:
Ativos (0): 73.46%
Evadiram (1): 26.54%

AVISO: O desequilíbrio de classes é significativo.
A classe minoritária (Evadiram) representa menos de 30% do total de clientes.
Isso pode afetar o desempenho dos modelos preditivos, fazendo-os tender a prever a classe majoritária.


### --- Divisão dos Dados em Treino e Teste ---

In [6]:
from sklearn.model_selection import train_test_split
from collections import Counter

# --- Divisão dos Dados em Treino e Teste ---
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print("Distribuição das classes ANTES do balanceamento (apenas nos dados de treino):")
print(f"Classe 'Não Evadiu' (0): {Counter(y_treino)[0]}")
print(f"Classe 'Evadiu' (1): {Counter(y_treino)[1]}")
print("-" * 50)

Distribuição das classes ANTES do balanceamento (apenas nos dados de treino):
Classe 'Não Evadiu' (0): 3880
Classe 'Evadiu' (1): 1402
--------------------------------------------------


### ---  Padronização dos Dados Numéricos ---

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Preencher valores faltantes nos dados de treino e teste
# O SimpleImputer retorna um array numpy, vamos usá-lo para alimentar o SMOTE
imputer = SimpleImputer(strategy='mean')
X_treino_imputado = imputer.fit_transform(X_treino)
X_teste_imputado = imputer.transform(X_teste)

# Identificando as colunas numéricas que precisam ser padronizadas
colunas_numericas = ['Meses_Permanencia', 'Cobranca_Mensal', 'Cobranca_Total']
# Obtendo os índices dessas colunas, pois o X_treino_imputado é um array numpy
indices_numericos = [X.columns.get_loc(col) for col in colunas_numericas]

# Criando o scaler
scaler = StandardScaler()

# Aplicar o fit_transform Apenas nos índices das colunas numéricas do conjunto de treino
X_treino_imputado[:, indices_numericos] = scaler.fit_transform(X_treino_imputado[:, indices_numericos])

# Aplicar o transform com os mesmos parâmetros ao conjunto de teste
X_teste_imputado[:, indices_numericos] = scaler.transform(X_teste_imputado[:, indices_numericos])

print("Dados de treino imputados e padronizados prontos para o balanceamento:")
print(X_treino_imputado[:5])
print("\nDados de teste imputados e padronizados prontos para a avaliação do modelo:")
print(X_teste_imputado[:5])



Dados de treino imputados e padronizados prontos para o balanceamento:
[[ 0.          0.51132396  1.02630142  0.94439029  1.          1.
   1.          1.          0.          1.          1.          0.
   0.          0.          0.          1.          0.          1.
   0.          0.          0.          0.          0.          1.
   1.          0.          1.          0.          0.          1.        ]
 [ 0.         -1.27780567  0.149951   -0.98024095  1.          0.
   0.          1.          0.          0.          1.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   1.          0.          1.          1.          0.          0.        ]
 [ 0.          1.16191656  0.71034965  1.25966735  0.          0.
   0.          1.          0.          1.          0.          0.
   0.          1.          0.          0.          0.          1.
   0.          1.          0.          1.          0.

### Balanceamento de Classes 

In [11]:
# --- Balanceamento com SMOTE (Oversampling) ---
from imblearn.over_sampling import SMOTE

print("\n" + "="*50)
print("Balanceamento das classes:")
print("="*50)
print("Distribuição das classes ANTES do balanceamento (apenas nos dados de treino):")
print(f"Classe 'Não Evadiu' (0): {Counter(y_treino)[0]}")
print(f"Classe 'Evadiu' (1): {Counter(y_treino)[1]}")
print("-" * 50)

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_treino_imputado, y_treino)

print("Distribuição das classes APÓS SMOTE (Oversampling):")
print(f"Classe 'Não Evadiu' (0): {Counter(y_smote)[0]}")
print(f"Classe 'Evadiu' (1): {Counter(y_smote)[1]}")
print("-" * 50)


Balanceamento das classes:
Distribuição das classes ANTES do balanceamento (apenas nos dados de treino):
Classe 'Não Evadiu' (0): 3880
Classe 'Evadiu' (1): 1402
--------------------------------------------------
Distribuição das classes APÓS SMOTE (Oversampling):
Classe 'Não Evadiu' (0): 3880
Classe 'Evadiu' (1): 3880
--------------------------------------------------
